In [1]:
! pip install -q kaggle
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gaurangdangayach","key":"6258e0a532bce00c861188066164a474"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              229KB  2021-06-19 20:05:51           7866  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4789  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1615  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   1GB  2021-06-09 15:58:24           2261  
mathurinache/twitter-edge-nodes                               Tw

In [5]:
! kaggle competitions download -c tabular-playground-series-jun-2021

  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 39.4MB/s]
  0% 0.00/6.49M [00:00<?, ?B/s]
100% 6.49M/6.49M [00:00<00:00, 59.6MB/s]
  0% 0.00/3.14M [00:00<?, ?B/s]
100% 3.14M/3.14M [00:00<00:00, 104MB/s]


In [6]:
! mkdir train
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [7]:
import torch
from torchvision import datasets, transforms

In [8]:
import pandas as pd
import numpy as np

In [13]:
train=pd.read_csv("train/train.csv")
import torch.utils.data as data_utils
train['target']=train['target'].map({'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3,'Class_5':4,'Class_6':5,'Class_7':6,'Class_8':7,'Class_9':8})
train_target = torch.tensor(train['target'].values)
train = torch.tensor(train.drop(['target','id'], axis = 1).values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train, train_target) 
trainloader = torch.utils.data.DataLoader(train_tensor, batch_size=64, shuffle=True)

In [14]:
vars, label = next(iter(trainloader))
label

tensor([1, 1, 7, 5, 8, 7, 7, 7, 6, 2, 5, 2, 8, 2, 7, 1, 7, 7, 7, 1, 6, 7, 7, 5,
        7, 3, 7, 5, 7, 7, 6, 5, 6, 0, 7, 8, 6, 7, 5, 1, 1, 5, 5, 7, 7, 6, 7, 7,
        5, 1, 1, 7, 7, 5, 7, 7, 5, 6, 7, 2, 7, 2, 7, 8])

In [15]:
from torch import nn, optim
import torch.nn.functional as F

In [16]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(75,24)
    self.fc2=nn.Linear(24,24)
    self.output=nn.Linear(24,9)

  def forward(self,x):
    h1=F.relu(self.fc1(x))
    h1=F.relu(self.fc2(h1))
    out=F.log_softmax(self.output(h1),dim=1)
    return out

In [17]:
model=Classifier()
criterion=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.05)

In [18]:
optimizer=optim.SGD(model.parameters(),lr=0.002)

In [21]:
! mkdir test
! unzip test.csv.zip -d train

Archive:  test.csv.zip
  inflating: train/test.csv          


In [22]:
test=pd.read_csv("train/test.csv")
test = torch.tensor(test.drop(['id'], axis = 1).values.astype(np.float32)) 
#test_tensor = data_utils.TensorDataset(train, train_target) 
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [24]:
epoch=10
for e in range(epoch):
  total_loss=0
  for vars,labels in iter(trainloader):
    log_p=model.forward(vars)
    loss=criterion(log_p,labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    optimizer.zero_grad()
  print(total_loss/len(trainloader))
  '''for test_vars in testloader:
    ttl=0
    test_pred=model.forward(test)
    test_loss=criterion(test_pred,labels)
    ttl=ttl+test_loss.item()
  print(ttl)'''

1.8683075304031371
1.8503391425704956
1.8371619162750243
1.82738032081604
1.819842695465088
1.8138731100082397
1.8090511811065675
1.8049663136291505
1.8014511330032348
1.7984777391052247


In [25]:
test=pd.read_csv("train/test.csv")
test = torch.tensor(test.drop(['id'], axis = 1).values.astype(np.float32)) 
#test_tensor = data_utils.TensorDataset(train, train_target) 
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [26]:
test_pred=model.forward(test)
pred_prob=torch.exp(test_pred)
pred_prob[0:5]

tensor([[0.0444, 0.2922, 0.1246, 0.0245, 0.0183, 0.2063, 0.0407, 0.1177, 0.1313],
        [0.0493, 0.1143, 0.0738, 0.0322, 0.0247, 0.2446, 0.0720, 0.2646, 0.1244],
        [0.0406, 0.0819, 0.0561, 0.0228, 0.0147, 0.4060, 0.0557, 0.2123, 0.1098],
        [0.0477, 0.0809, 0.0608, 0.0329, 0.0251, 0.2656, 0.0749, 0.2909, 0.1212],
        [0.0388, 0.0842, 0.0551, 0.0200, 0.0130, 0.3929, 0.0590, 0.2244, 0.1126]],
       grad_fn=<SliceBackward>)

In [27]:
pred_prob=pred_prob.detach().numpy()

In [28]:
columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']

df= pd.DataFrame(pred_prob,
                     columns=columns)

In [29]:
testing=pd.read_csv("train/test.csv")
df1=pd.concat([testing.iloc[:,0],df], axis=1)
df1.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.044404,0.292183,0.124595,0.024466,0.018288,0.206325,0.040729,0.117726,0.131285
1,200001,0.049322,0.114282,0.073830,0.032195,0.024694,0.244639,0.072029,0.264648,0.124360
2,200002,0.040648,0.081868,0.056144,0.022791,0.014684,0.405979,0.055744,0.212313,0.109828
3,200003,0.047680,0.080939,0.060769,0.032879,0.025134,0.265621,0.074927,0.290900,0.121152
4,200004,0.038800,0.084218,0.055150,0.020038,0.012953,0.392866,0.058977,0.224439,0.112560
